In [4]:
import sqlite3
import pandas as pd

In [5]:
conn = sqlite3.connect('/home/greenman225/Code/github/donkey_golf/donkey_golf/site.db')
pd.read_sql('SELECT name FROM sqlite_master WHERE type = "table"', conn)

,name
0,user
1,teams
2,rankings
3,pre_tourney
4,leaderboard


In [ ]:
sql = """
SELECT id, username FROM user
"""
pd.read_sql(sql, conn)

In [ ]:
sql = """
SELECT * FROM teams
"""
pd.read_sql(sql, conn)

In [ ]:
user_df = pd.read_sql(sql, conn)

In [ ]:
user_df

In [45]:
def aggregate_team_score():
    '''
    Pull each users aggregate score for the current tourney
    '''
    
    sql = """
    SELECT c.id, c.username, b.* FROM teams a
    LEFT JOIN leaderboard b 
    ON a.golfer=b.player
    LEFT JOIN user c 
    ON a.id=c.id
    """

    df = pd.read_sql(sql, conn)

    # Convert to par field to an integer so we can aggregate
    df['user_score'] = df['to_par'].str.replace('+','').str.replace('E','0')
    df['user_score'] = df['user_score'].astype(int)

    # Get an overall score for each user
    team_score_df = df.groupby(['id','username'])['user_score'].sum().reset_index()
    team_score_df.sort_values('user_score', inplace=True)
    
    team_score_df['rank'] = team_score_df['user_score'].rank(ascending=True, method='min').astype(int)

    return team_score_df

def determine_ties_in_scoreboard(df):
    '''
    Takes a scoreboard df and determines rank and handles ties
    '''
    df_ties = df.copy()

    # Need to create a better rank that handles ties
    df_ties['new_rank'] = df_ties['rank'].copy()

    df_ties = df_ties.groupby(['rank'])['id'].count().reset_index()
    df_ties.loc[df_ties['id'] > 1, 'new_rank'] = "T" 

    df_ties.loc[df_ties['new_rank'].notnull(), 'final_rank'] = 'T' + df_ties['rank'].astype(str)
    df_ties.loc[df_ties['new_rank'].isnull(), 'final_rank'] = df_ties['rank'].astype(str)
    
    return df_ties[['rank','final_rank']]

def pull_scoreboard():
    '''
    Pulls aggregate game score and then adds in tie info
    '''
    df = pull_game_score()
    tie_df = determine_ties_in_scoreboard(df)
    
    df = df.merge(tie_df,
        how='left',
        on='rank')
    
    return df

In [46]:
df = pull_game_score()

In [55]:
pull_scoreboard()

,id,username,user_score,rank,final_rank
0,7,tiger,-28,1,T1
1,9,donk,-28,1,T1
2,6,jeff,-3,3,3
3,4,poo,7,4,4
4,5,mike,9,5,5


In [51]:
df.merge(tie_df,
        how='left',
        on='rank')

,id,username,user_score,rank,final_rank
0,7,tiger,-28,1,T1
1,9,donk,-28,1,T1
2,6,jeff,-3,3,3
3,4,poo,7,4,4
4,5,mike,9,5,5


In [40]:
df_ties

,rank,id,new_rank,final_rank
0,1,2,T,T1
1,3,1,NaN,3
2,4,1,NaN,4
3,5,1,NaN,5


In [31]:

#+ df_ties.loc[df_ties['id'] > 1, 'rank'].astype(str)
#df_ties = df_ties[['rank', 'new_rank']].copy()

In [32]:
df_ties

,rank,id,new_rank
0,1,2,T
1,3,1,NaN
2,4,1,NaN
3,5,1,NaN


In [34]:
df_ties.loc[df_ties['new_rank'].notnull(), 'final_rank'] = 'T' + df_ties['rank'].astype(str)

df_ties.loc[df_ties['new_rank'].isnull(), 'final_rank'] = df_ties['rank'].astype(str)

In [38]:
df_ties

,rank,id,new_rank,final_rank
0,1,2,T,T1
1,3,1,NaN,3
2,4,1,NaN,4
3,5,1,NaN,5


In [20]:
df

,id,username,user_score,rank
3,7,tiger,-28,1
4,9,donk,-28,1
2,6,jeff,-3,3
0,4,poo,7,4
1,5,mike,9,5


In [ ]:
pd.read_sql("SELECT * FROM leaderboard", conn)

# Show all tables in our database 

In [ ]:
pd.read_sql('SELECT name FROM sqlite_master WHERE type = "table"', conn)

In [ ]:
user_id=6
sql = f"""
SELECT * from teams a
INNER JOIN leaderboard b
    ON a.golfer = b.player
WHERE id = {user_id}    
"""
pd.read_sql(sql, conn)

In [ ]:
pd.read_sql('SELECT * from leaderboard', conn)

In [ ]:
pd.read_sql('SELECT * from user', conn)